## Reading the data from PostgreSQL database and looking at it with psycopg2

In [3]:
import pandas as pd
import os
os.chdir('/Users/beth/Documents/Metis/metis_project_3_files/clinical_trials')

In [4]:
import psycopg2 as pg

In [5]:
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'clinical_trials',    # DB that we are connecting to
}

In [6]:
conn = pg.connect(**connection_args)

In [7]:
def go(query):
    return pd.read_sql_query(query, conn)

In [8]:
query = "SELECT * FROM basic_info LIMIT 2;"

pd.read_sql_query(query, conn)

,idx,rank,nctid,brieftitle,condition,keyword,overallstatus,whystopped,startdate,primarycompletiondate,completiondate,referencepmid,referencetype,isfdaregulateddrug,isfdaregulateddevice,isunapproveddevice,hasexpandedaccess,isusexport,oversighthasdmc
0,0,1,NCT00049660,Capecitabine Compared With Vinorelbine in Trea...,Breast Cancer,stage IV breast cancer,Terminated,low accrual,September 2002,December 2004,,17976988,result,,,,No,,
1,1,2,NCT00503321,Phase II Study of TS-1 Therapy and TS-1+PSK Th...,Gastric Cancer,"gastric cancer, TS-1, PSK",Terminated,Patients' enrollment was not sufficient.,October 2006,August 2009,August 2011,7910230,background,,,,No,,Yes


In [9]:
query = "SELECT COUNT(*) FROM basic_info;"
go(query)

,count
0,18125


In [10]:
query = "SELECT COUNT(*) FROM basic_info WHERE overallstatus='Terminated';"
go(query)

,count
0,3810


In [11]:
query = "SELECT COUNT(*) FROM basic_info WHERE overallstatus='Completed';"
go(query)

,count
0,14315


In [12]:
query = "SELECT overallstatus, COUNT(*) FROM basic_info WHERE hasexpandedaccess = 'Yes' \
GROUP BY overallstatus;"

go(query)

,overallstatus,count
0,Completed,51
1,Terminated,13


In [13]:
query = "SELECT to_date(startdate,'Month YYYY') from basic_info LIMIT 2;"

go(query)

,to_date
0,2002-09-01
1,2006-10-01


In [14]:
#there are some studies with no entry for completion data
query = "SELECT COUNT(primarycompletiondate) FROM basic_info WHERE primarycompletiondate = '';"
go(query)

,count
0,1457


In [15]:
#there are some studies with no entry for completion data
query = "SELECT COUNT(primarycompletiondate) FROM basic_info WHERE primarycompletiondate != '';"
go(query)

,count
0,16668


In [16]:
query = "SELECT startdate as start, primarycompletiondate \
as stop FROM basic_info WHERE primarycompletiondate != '' AND startdate != '';"
dates = go(query)
dates

,start,stop
0,September 2002,December 2004
1,October 2006,August 2009
2,May 2006,July 2008
3,August 2007,October 2011
4,December 2006,December 2010
...,...,...
16641,February 2007,December 2009
16642,March 2012,August 2012
16643,July 2012,"May 22, 2015"
16644,"September 11, 2018","August 26, 2019"


In [17]:
study_length = pd.to_datetime(dates['stop'])-pd.to_datetime(dates['start'])
study_length.median()

Timedelta('1187 days 00:00:00')

In [18]:
query = "SELECT * FROM study_design LIMIT 4;"

pd.read_sql_query(query, conn)

,idx,rank,nctid,designprimarypurpose,phase,designinterventionmodel,designinterventionmodeldescription,designwhomasked,designallocation,enrollmentcount,enrollmenttype,armgrouplabel,armgrouptype,armgroupdescription,interventiontype,interventionname,interventionothername,interventiondescription
0,0,1,NCT00049660,Treatment,Phase 2|Phase 3,,,,Randomized,47.0,Actual,,,,Drug|Drug,capecitabine|vinorelbine tartrate,,
1,1,2,NCT00503321,Treatment,Phase 2|Phase 3,Parallel Assignment,,,Randomized,13.0,Actual,Arm B|Arm A,Experimental|Active Comparator,S-1 plus PSK group|S-1 alone,Drug|Drug,"Tegafur/gimeracil/oteracil potassium (S-1), Kr...",S-1 plus PSK|S-1 alone,"S-1 80mg/m2 4weeks on 2 weeks off, PSK 3g/day|..."
2,2,3,NCT00312442,Treatment,Phase 2|Phase 3,Single Group Assignment,,,Non-Randomized,16.0,Actual,WST 09,Experimental,Treatment with WST09-mediated VTP,Drug,Treatment with WST09 Vascular Photodynamic the...,,
3,3,4,NCT00507429,Treatment,Phase 2|Phase 3,Parallel Assignment,,,Randomized,80.0,Actual,Arm 1: CA4P + Carboplatin + paclitaxel|Arm 2: ...,Experimental|Active Comparator,"Six 21-day cycles: CA4P (60 mg/m2 on Days 1, 8...",Drug|Drug|Drug,CA4P|paclitaxel|carboplatin,combretastatin|fosbretabulin|Zybrestat|Taxol|P...,"CA4P 60mg/m squared for Days 1, 8, 15 for 6 cy..."


In [19]:
query = "WITH temp as \
(SELECT basic_info.overallstatus as overallstatus, study_design.designinterventionmodel as dim \
FROM basic_info LEFT JOIN study_design ON basic_info.nctid = study_design.nctid) \
SELECT dim, COUNT(*) FROM temp WHERE overallstatus = 'Terminated' \
GROUP BY dim;"

go(query)

,dim,count
0,Parallel Assignment,1380
1,Crossover Assignment,40
2,Sequential Assignment,42
3,,160
4,Factorial Assignment,18
5,Single Group Assignment,2170


In [20]:
query = "WITH temp as \
(SELECT basic_info.overallstatus as overallstatus, study_design.designinterventionmodel as dim \
FROM basic_info LEFT JOIN study_design ON basic_info.nctid = study_design.nctid) \
SELECT dim, COUNT(*) FROM temp WHERE overallstatus = 'Completed' \
GROUP BY dim;"

go(query)

,dim,count
0,Parallel Assignment,5150
1,Crossover Assignment,215
2,Sequential Assignment,99
3,,1582
4,Single Group Assignment,7155
5,Factorial Assignment,114


I really don't understand why the below query failed yet

In [35]:
query = "SELECT designprimarypurpose, SUM(CAST(enrollmentcount AS DOUBLE PRECISION)) as total \
FROM study_design WHERE enrollmentcount != '' \
GROUP BY designprimarypurpose ORDER BY total DESC;"

go(query)

,designprimarypurpose,total
0,Treatment,2219652.0
1,Prevention,356227.0
2,Screening,113228.0
3,Supportive Care,97896.0
4,Diagnostic,68431.0
5,,21885.0
6,Health Services Research,6131.0
7,Other,4996.0
8,Basic Science,2256.0
9,Educational/Counseling/Training,1310.0


In [50]:
query = "SELECT designprimarypurpose, SUM(CAST(enrollmentcount AS DOUBLE PRECISION)) \
FROM study_design WHERE enrollmentcount != '' \
GROUP BY designprimarypurpose HAVING SUM(CAST(enrollmentcount AS DOUBLE PRECISION))\
 > 10000.0;"

go(query)

,designprimarypurpose,sum
0,Diagnostic,68431.0
1,Prevention,356227.0
2,Treatment,2219652.0
3,,21885.0
4,Supportive Care,97896.0
5,Screening,113228.0


In [55]:
query = "SELECT armgrouptype, COUNT(*) \
FROM study_design WHERE interventionname LIKE '%paclitaxel%' \
GROUP BY armgrouptype ORDER BY COUNT(*) DESC LIMIT 4;"

go(query)

,armgrouptype,count
0,,172
1,Experimental,159
2,Experimental|Experimental,45
3,Experimental|Active Comparator,31


In [57]:
cursor = conn.cursor()

In [61]:
cursor.execute("SELECT condition, whystopped FROM basic_info WHERE overallstatus = 'Terminated' AND \
 whystopped LIKE '%accrual%';")

In [62]:
cursor.fetchone()

('Breast Cancer', 'low accrual')

In [63]:
cursor.fetchone()

('Anaplastic Thyroid Cancer', 'Low rate of subject accrual')

In [64]:
cursor.fetchmany(3)

[('Post-operative Pneumonia|Lung Cancer|Esophageal Cancer',
  'Slow patient accrual and plans to perform multi-center study'),
 ('Anal Cancer', 'low accrual'),
 ('Advanced Cancers', 'Terminated due to low accrual.')]

In [65]:
cursor.execute("SELECT condition, whystopped FROM basic_info WHERE overallstatus = 'Terminated' AND \
 whystopped LIKE '%safety%';")

In [67]:
cursor.fetchmany(3)

[('Breast Cancer',
  'Terminated: Amgen decision following Primary Analysis. Not due to safety reason'),
 ('Pain|Neoplasms|Chronic Pain',
  'low accrual made the study no longer feasible/decision not related to safety and efficacy'),
 ('Prostatic Neoplasms',
  'Study A6181120 was prematurely discontinued due to futility on 27 September 2010. No new or unexpected safety issues were identified.')]